In [32]:
import pandas as pd


In [33]:
def extract_sentence(df):
    
    for index, row in df.iterrows():
        player = row['player']
        data = str(row['data'])  # Convert data to string
        
        # Find the sentence containing player's information
        sentences = data.split('.')
        for sentence in sentences:
            if player.lower() in sentence.lower():
                df.at[index, 'data'] = sentence.strip()
                break
    
    return df

In [34]:
# Load the CSV file into a DataFrame
df_de = pd.read_csv('../Preprocessing/data_clean/de_clean_1.csv')
df_en = pd.read_csv('../Preprocessing/data_clean/en_clean_1.csv')
df_es = pd.read_csv('../Preprocessing/data_clean/es_clean_1.csv')

In [35]:
df_de = extract_sentence(df_de)
df_en = extract_sentence(df_en)
df_es = extract_sentence(df_es)

In [36]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'de_clean_1-1.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
df_de.to_csv(file_path, index=False)

In [37]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'en_clean_1-1.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
df_en.to_csv(file_path, index=False)

In [38]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'es_clean_1-1.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
df_es.to_csv(file_path, index=False)